# BlackBox Matching

### Import Statement

In [108]:
import Conf_Calculator as CC
import pandas as pd 
import string_sum as sr

### Utility Functions

In [109]:
def ConfigurationBoundleString(list_of_items):
    exit_string = ''
    for elem_pos in range(len(list_of_items)):
        if (elem_pos + 1)%2 == 0:
            exit_string = exit_string + '~' + str(list_of_items[elem_pos])
        else:
            exit_string = exit_string + ',' + str(list_of_items[elem_pos])
    
    return exit_string[1:]

In [110]:
def getOrderDat_returnBoundleToOrder(dataFrameOfOrders):
    string_out = ''
    list_storing_strings = []
    months = []
    seq_numbers = []
    month = None
    OrderSeq = None
    for  index, row in dataFrameOfOrders.iterrows():

        #first line
        if (month is None) and (OrderSeq is None):
            string_out = row['SKU'] + '~' + str(row['qty'])
            month, OrderSeq = row['Month'], row['OrdSeq']

        elif (month != row['Month']) or( OrderSeq != row['OrdSeq']):
            list_storing_strings.append(string_out)
            months.append(month)
            seq_numbers.append(OrderSeq)
            
            string_out = row['SKU'] + '~' + str(row['qty'])
            month, OrderSeq = row['Month'], row['OrdSeq'] 
        else:
            string_out = string_out + str(row['SKU']) + '~' + str(row['qty']) + ',' 

    d = {'boundles': list_storing_strings, 'month': months, 'seqNumb':seq_numbers }
    boundlesToMonthSquenceOrders = pd.DataFrame(data=d)     
    return boundlesToMonthSquenceOrders

### Process

In [143]:
OCpR, Prices, Orders = CC.dataframes_splitter()

In [170]:
finalizedModel = CC.loop_thru_dataframes(OCpR)
finalizedModel = finalizedModel[ list(filter( lambda x : 'tag' not in x,  finalizedModel.columns)) ]

In [146]:
config_boundles = pd.DataFrame()
config_boundles['code'] = finalizedModel.index
config_boundles['concat'] = pd.Series(finalizedModel.fillna('').values.tolist()).map(lambda x: ConfigurationBoundleString(x))

In [147]:
dataFrameOfOrders = Orders[0]

In [148]:
boundlesToMonthSquenceOrders = getOrderDat_returnBoundleToOrder(dataFrameOfOrders)

In [149]:
conf_list_of_boundles_data = list(map(list,zip(config_boundles['concat'], config_boundles['code'].astype(str))))

In [150]:
boundlesToMonthSquenceOrders_data = list(map(list,zip(  boundlesToMonthSquenceOrders['boundles'], boundlesToMonthSquenceOrders['month'].astype(str)+ "_" + boundlesToMonthSquenceOrders['seqNumb'].astype(str)  )))


In [151]:
b = sr.loop_two_lists_rayon_codes(conf_list_of_boundles_data , 
                                  boundlesToMonthSquenceOrders_data, 0.8)


In [152]:
fictionary_matching_results = {}
for i in b:
    if i.orderseq in fictionary_matching_results.keys():
        if (i.score > fictionary_matching_results[i.orderseq][1]):
            fictionary_matching_results[i.orderseq] =  [i.boundle_code, i.score]
    else:
        fictionary_matching_results[i.orderseq] =  [i.boundle_code, i.score]

In [161]:
month = []
seq   = []
match = []
for match_ in fictionary_matching_results.keys():
    l_identity_order = match_.split('_') 
    month.append(int(l_identity_order[0]))
    seq.append(int(l_identity_order[1]))
    match.append(fictionary_matching_results[match_][0])
d = {'Month': month, 'OrdSeq': seq, 'match': match}
matched_df = pd.DataFrame(data=d)

In [163]:
dataFrameOfOrders['unit_Price_QTY'] = dataFrameOfOrders['unit_Price'] * dataFrameOfOrders['qty']

In [164]:
orders_totals = dataFrameOfOrders[['Month', 'OrdSeq', 'unit_Price_QTY']].groupby(['Month', 'OrdSeq']).sum().reset_index()

In [208]:
orders_match_prices = pd.merge( matched_df, orders_totals, on=['Month', 'OrdSeq'],how='inner')
orders_match_prices.rename(columns={'match':'code'}, inplace=True)
orders_match_prices['code'] = orders_match_prices['code'].astype(int)

In [209]:
OCpR, Prices, Orders = CC.dataframes_splitter()
Model_Tags = CC.loop_thru_dataframes(OCpR)
Model_Tags = Model_Tags[ list(filter( lambda x : 'tag' in x,  Model_Tags.columns))  ]
Model_Tags['code'] = Model_Tags.index
Model_Tags['code'] = Model_Tags['code'].astype(int)

In [212]:
analyzed_orders = pd.merge(orders_match_prices, Model_Tags, on=['code'])

In [213]:
analyzed_orders.head()

,Month,OrdSeq,code,unit_Price_QTY,CPU_level_tag,cache_limit_tag,vSAN_level_tag,N_years_service_tag
0,1,5,34,5539.8081,Platinum,single,Enterprise,3
1,2,14,34,5419.3590,Platinum,single,Enterprise,3
2,3,35,34,5337.2475,Platinum,single,Enterprise,3
3,1,6,26,5277.4788,Platinum,single,Enterprise,3
4,4,17,26,5084.5275,Platinum,single,Enterprise,3
